In [24]:
%reset -f

In [25]:

import pandas as pd
import numpy as np
from pathlib import Path
import ssl
from email.message import EmailMessage
import os
import smtplib

bases_path = Path("bases")
listagem_path = Path("listagem")

output = Path("OUTPUT")
output.mkdir(exist_ok=True)

relatorio = bases_path / "relatorio_ra-todas_03e04-12-2025.xlsx"
listagem = listagem_path / "listagem.xlsx"
df_relatorio  = pd.read_excel(relatorio, engine="calamine", header=5, sheet_name='Relatório')
df_lista = pd.read_excel(listagem, engine="calamine")



In [26]:
df_relatorio.head(2)

,Loja,Cod. Loja,CNPJ,CLISAP,Grife,Artigo,Cor,Tamanho,Sexo,Grupo,...,Faltou no Estoque,Atendido (SAP),Valor,Nr. Pedido,Data Pedido,Data Embarque,Data de Geração,Status Item,Status Pedido,Retorno Pedido
0,RUA ANDRE CAVALCANTI-BHT,100,4729921000140,3612,HA,0111,AX7EN,M,MASCULINO,REGATAS,...,0,2,49,0026764562,2025-12-03 17:28:02,2025-12-15 00:00:00,2025-12-03,Sucesso,PEDACE,Venda Programada 26764562 foi gravado(a)
1,RUA ANDRE CAVALCANTI-BHT,100,4729921000140,3612,HA,0111,AX7EN,P,MASCULINO,REGATAS,...,0,2,49,0026764562,2025-12-03 17:28:02,2025-12-15 00:00:00,2025-12-03,Sucesso,PEDACE,Venda Programada 26764562 foi gravado(a)


In [27]:
df_relatorio.columns

Index(['Loja', 'Cod. Loja', 'CNPJ', 'CLISAP', 'Grife', 'Artigo', 'Cor',
       'Tamanho', 'Sexo', 'Grupo', 'Linha', 'Vel. de Vendas', 'Lead Time',
       'Cobertura', 'Mínimo', 'Alvo', 'Em Loja', 'Em Transito', 'Em Carteira',
       'Necessidade', 'Ruptura', 'Ressuprir', 'Cancelada', 'Faltou no Estoque',
       'Atendido (SAP)', 'Valor', 'Nr. Pedido', 'Data Pedido', 'Data Embarque',
       'Data de Geração', 'Status Item', 'Status Pedido', 'Retorno Pedido'],
      dtype='object')

In [28]:
df_relatorio['Sem_Estoque'] = (df_relatorio['Ruptura'] + df_relatorio['Faltou no Estoque']).abs()

In [29]:
df_relatorio['Sem_Estoque'].sum()

np.int64(2568)

In [30]:
#df_relatorio = df_relatorio.drop(columns='Unnamed: 33')

In [31]:
df_relatorio.dtypes

Loja                  object
Cod. Loja              int64
CNPJ                   int64
CLISAP                 int64
Grife                 object
Artigo                object
Cor                   object
Tamanho               object
Sexo                  object
Grupo                 object
Linha                 object
Vel. de Vendas       float64
Lead Time            float64
Cobertura              int64
Mínimo                 int64
Alvo                   int64
Em Loja                int64
Em Transito            int64
Em Carteira            int64
Necessidade            int64
Ruptura                int64
Ressuprir              int64
Cancelada              int64
Faltou no Estoque      int64
Atendido (SAP)         int64
Valor                 object
Nr. Pedido            object
Data Pedido           object
Data Embarque         object
Data de Geração       object
Status Item           object
Status Pedido         object
Retorno Pedido        object
Sem_Estoque            int64
dtype: object

In [32]:
df_lista.head()

,Cod. Loja,Nome loja,Franqueado
0,3,HERING ALAMEDA BARROS,AURIDENICHE STEFANO
1,5,HS FREI CANECA,ARLINDO DE OLIVEIRA LIMA FILHO
2,7,HERING PAMPLONA,MARCO ANTONIO SIMOES CHADAD
3,9,RUA AUGUSTA,ANDRE HERING
4,10,HERING ACLIMACAO,LUIZ ANTONIO LUCIANO


In [33]:
df_lista.dtypes

Cod. Loja      int64
Nome loja     object
Franqueado    object
dtype: object

In [34]:
df_lista.columns

Index(['Cod. Loja', 'Nome loja', 'Franqueado'], dtype='object')

In [35]:
df_relatorio = df_relatorio.merge(df_lista[['Cod. Loja', 'Franqueado']], how='left', on='Cod. Loja')

In [36]:
df_relatorio['Franqueado'].head()

0    CARLOS RODRIGUES MENDES DA SILVA
1    CARLOS RODRIGUES MENDES DA SILVA
2    CARLOS RODRIGUES MENDES DA SILVA
3    CARLOS RODRIGUES MENDES DA SILVA
4    CARLOS RODRIGUES MENDES DA SILVA
Name: Franqueado, dtype: object

In [37]:
df_relatorio.isnull().sum()

Loja                 0
Cod. Loja            0
CNPJ                 0
CLISAP               0
Grife                0
Artigo               0
Cor                  0
Tamanho              0
Sexo                 0
Grupo                0
Linha                0
Vel. de Vendas       0
Lead Time            0
Cobertura            0
Mínimo               0
Alvo                 0
Em Loja              0
Em Transito          0
Em Carteira          0
Necessidade          0
Ruptura              0
Ressuprir            0
Cancelada            0
Faltou no Estoque    0
Atendido (SAP)       0
Valor                0
Nr. Pedido           0
Data Pedido          0
Data Embarque        0
Data de Geração      0
Status Item          0
Status Pedido        0
Retorno Pedido       0
Sem_Estoque          0
Franqueado           0
dtype: int64

In [38]:
print("validação 1...")
pedidos_sem_retorno = df_relatorio[df_relatorio['Retorno Pedido'] == '-']
condicao_1 = (pedidos_sem_retorno['Ressuprir'].sum() > 0) or (pedidos_sem_retorno['Atendido (SAP)'] > 0).any()

if condicao_1:
    print("!! ALERTA: Há pedidos sem retorno, com 'Ressuprir' ou 'Atendido maior que 0")
else: 
    print("OK: Não há pedidos sem retorno com 'Ressuprir' ou 'Atendido' maior que 0")

print("-"*50)

print("Validação 2...")

filtro_cancelada = df_relatorio['Cancelada'] == 0
filtro_atendido_sap = df_relatorio['Atendido (SAP)'] == 0
filtro_necessidade = df_relatorio['Necessidade'] > df_relatorio['Sem_Estoque']

pedidos_inconsistentes = df_relatorio[filtro_cancelada & filtro_atendido_sap & filtro_necessidade]

if not pedidos_inconsistentes.empty:
    print("!! ALERTA: Há pedidos com necessidade maior que Sem_Estoque, não foram atendidos e não constam como cancelados")
else:
    print("OK: Todos os pedidos com necessidade maior que o estoque foram tratados corretamente")

    

validação 1...
OK: Não há pedidos sem retorno com 'Ressuprir' ou 'Atendido' maior que 0
--------------------------------------------------
Validação 2...
!! ALERTA: Há pedidos com necessidade maior que Sem_Estoque, não foram atendidos e não constam como cancelados


In [39]:
colunas_para_soma = ['Necessidade', 'Sem_Estoque', 'Ressuprir', 'Atendido (SAP)']

filtro_cancelada = df_relatorio['Cancelada'] == 0
filtro_atendido_sap = df_relatorio['Atendido (SAP)'] == 0
filtro_necessidade = df_relatorio['Necessidade'] > df_relatorio['Sem_Estoque']

pedidos_com_problemas = df_relatorio[filtro_cancelada & filtro_atendido_sap & filtro_necessidade].copy()

for col in colunas_para_soma:
    pedidos_com_problemas[col] = pd.to_numeric(pedidos_com_problemas[col], errors="coerce").fillna(0)


TD_PROBLEM = pedidos_com_problemas.groupby(['Cod. Loja', 'Artigo', 'Cor', 'Tamanho'])[colunas_para_soma].sum()
TD_PROBLEM = TD_PROBLEM.reset_index()
TD_PROBLEM

,Cod. Loja,Artigo,Cor,Tamanho,Necessidade,Sem_Estoque,Ressuprir,Atendido (SAP)
0,218,KZGR,1ASN,M,4,1,4,0
1,272,KZGR,1ASN,M,3,1,3,0
2,552,H1MV,2DSN,042,3,1,3,0
3,552,N3A7,N0A00S,P,4,1,4,0
4,715,55KN,AXTEN,006,6,5,2,0
5,1000,55KN,AXTEN,008,5,2,4,0
6,1211,KZGR,1ASN,P,3,1,3,0
7,1292,KZGR,1ASN,P,2,1,2,0
8,1292,KZGR,1BSN,M,2,1,2,0


In [40]:
def formatar_moeda(valor):
    return f'R$ {valor:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')

def formatar_qtd(valor):
    return f'{valor:,.0f}'.replace(',', '.')

In [41]:
filtro = df_relatorio['Sem_Estoque'] > 0
df_filtrado = df_relatorio[filtro].copy()

TD_STK = df_filtrado.groupby(['Grife', 'Artigo', 'Cor', 'Tamanho'])[['Sem_Estoque']].sum().reset_index()


soma_total = TD_STK['Sem_Estoque'].sum()

# 3. Criar a linha de Total manualmente
# Como temos várias colunas de texto (Artigo, Cor, etc), precisamos decidir o que colocar nelas (deixei vazio '')
nova_linha = {
    'Grife': 'Total Geral',
    'Artigo': '',
    'Cor': '',
    'Tamanho': '',
    'Sem_Estoque': soma_total
}

# 4. Transformar em DataFrame e Concatenar
df_total_B = pd.DataFrame([nova_linha])
TD_STK = pd.concat([TD_STK, df_total_B], ignore_index=True)

TD_STK['Sem_Estoque'] = TD_STK['Sem_Estoque'].apply(formatar_qtd)
# Visualizar
TD_STK.head() # Mostra apenas o final para confirmar o total

,Grife,Artigo,Cor,Tamanho,Sem_Estoque
0,HA,036H,3REN,XG,52
1,HA,036H,3REN,XXG,32
2,HA,036H,5DEN,G,81
3,HA,036H,6UEN,XG,53
4,HA,036H,7QEN,G,96


In [42]:
cols_to_sum = ['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)', 'Valor']

for col in cols_to_sum:
    # 'coerce' transforma textos estranhos em NaN (vazio), depois preenchemos com 0
    df_relatorio[col] = pd.to_numeric(df_relatorio[col], errors='coerce').fillna(0)

    
TD_RESUMO = df_relatorio.groupby('Grife')[['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)', 'Valor']].sum()
TD_RESUMO = TD_RESUMO.reset_index()
TD_RESUMO

,Grife,Necessidade,Sem_Estoque,Cancelada,Atendido (SAP),Valor
0,HA,43421,2230,3974,37200,1525705.30
1,HI,2825,77,235,2493,75455.78
2,HK,3689,261,259,3165,99574.51


In [43]:
cols_numericas = ['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)', 'Valor']
TD_RESUMO = df_relatorio.groupby('Grife')[cols_numericas].sum().reset_index()

# 2. CALCULA O TOTAL GERAL
# Somamos apenas as colunas numéricas
linha_total = TD_RESUMO[cols_numericas].sum()
# Definimos o nome da primeira coluna para aparecer na linha de baixo
linha_total['Grife'] = 'Total Geral'

# 3. ADICIONA A LINHA DE TOTAL AO DATAFRAME
# Transformamos a série em DataFrame e concatenamos
df_total = pd.DataFrame([linha_total])
TD_RESUMO = pd.concat([TD_RESUMO, df_total], ignore_index=True)


# Aplica formatação de moeda na coluna Valor
TD_RESUMO['Valor'] = TD_RESUMO['Valor'].apply(formatar_moeda)

# Aplica formatação de quantidade nas outras colunas
cols_qtd = ['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)']
for col in cols_qtd:
    TD_RESUMO[col] = TD_RESUMO[col].apply(formatar_qtd)


display(TD_RESUMO)

,Grife,Necessidade,Sem_Estoque,Cancelada,Atendido (SAP),Valor
0,HA,43.421,2.230,3.974,37.200,"R$ 1.525.705,30"
1,HI,2.825,77,235,2.493,"R$ 75.455,78"
2,HK,3.689,261,259,3.165,"R$ 99.574,51"
3,Total Geral,49.935,2.568,4.468,42.858,"R$ 1.700.735,59"


In [44]:
filtro_cancelada = df_relatorio['Cancelada'] > 0
df_cancelado = df_relatorio[filtro_cancelada].copy()
TD_CANC = df_cancelado.groupby('Franqueado')[['Necessidade', 'Cancelada']].sum().reset_index()

total_nec = TD_CANC['Necessidade'].sum()
total_canc = TD_CANC['Cancelada'].sum()

linha_total = {
    'Franqueado': 'Total Geral',
    'Necessidade': total_nec,
    'Cancelada': total_canc
}


TD_CANC = pd.concat([TD_CANC, pd.DataFrame([linha_total])], ignore_index=True)

col_qtd = ['Necessidade', 'Cancelada']

for col in col_qtd:
    TD_CANC[col] = TD_CANC[col].apply(formatar_qtd)


TD_CANC

,Franqueado,Necessidade,Cancelada
0,ANA CRISTINA PUCCI,1.056,1.055
1,CANDIDA PIRES MARTINS NETA PEREIRA,175,175
2,MARCELO GROSSI KAMINSKI,269,269
3,MARIANA BRANDAO KAISER CABRAL LACOURT,262,259
4,NEIVA DONIZETE LOPES FERREIRA DE OLIVEIRA,166,166
5,REGINA OLIVEIRA LOPES,189,189
6,ROSANE FILOMENA ROCHA VIEIRA MARTINS,63,63
7,SHELON TEIXEIRA FORTINI,97,97
8,UBIRATAN JUN TANIMITSU HASEGAWA,2.208,2.195
9,Total Geral,4.485,4.468


In [45]:
cols_colc = ['Necessidade', 'Sem_Estoque', 'Cancelada', 'Atendido (SAP)', 'Valor']

for col in cols_colc:
    df_relatorio[col] = pd.to_numeric(df_relatorio[col], errors='coerce').fillna(0)


tot_necessidade = df_relatorio['Necessidade'].sum()
tot_sem_estoque = df_relatorio['Sem_Estoque'].sum()
tot_cancelado   = df_relatorio['Cancelada'].sum()
tot_atendido    = df_relatorio['Atendido (SAP)'].sum()
tot_receita     = df_relatorio['Valor'].sum()

# Cálculos de Porcentagem
pct_sem_estoque = (tot_sem_estoque / tot_necessidade * 100) if tot_necessidade > 0 else 0
pct_cancelado   = (tot_cancelado / tot_necessidade * 100) if tot_necessidade > 0 else 0


def fmt_br(valor, eh_moeda=False):
    if eh_moeda:
        texto = f'{valor:,.2f}' #Adiciona vírgula como separador de milhar e garante sempre 2 casas decimais.
        return texto.replace(',', 'X').replace('.', ',').replace('X', '.')
    else:
        texto = f'{valor:,.0f}'
        return texto.replace(',', '.')
    

    

In [ ]:
import sys # Importante para encerrar o script se a opção for inválida

print("\nSelecione o tipo de envio:")
print("1 - Franquia")
print("2 - Lojas Próprias (L.P.)")
opcao = input("Digite o número da opção desejada: ")

# Definição das listas de e-mails
emails_franquia = [
    "supervisorasfq@ciahering.com.br",
    "leticia.yuanaga@ciahering.com.br",
    "daniela.barboza@ciahering.com.br",
    "thais.zutter@ciahering.com.br",
    "anderson.puff@ciahering.com.br",
    "andrey.martins@ciahering.com.br",
    "joao.boll@ciahering.com.br",
    "giulia.freitas@ciahering.com.br",
    "paulo.saldanha@ciahering.com.br",
    "dayane.lemos@ciahering.com.br",
    "sabrina.cordeiro@ciahering.com.br"
]

emails_lp = [
    "giulia.freitas@ciahering.com.br",
    "anderson.puff@ciahering.com.br",
    "danilo.miguel@ciahering.com.br",
    "dayane.lemos@ciahering.com.br",
    "thais.zutter@ciahering.com.br",
    "andrey.martins@ciahering.com.br",
    "joao.boll@ciahering.com.br",
    "paulo.saldanha@ciahering.com.br"
]

# Lógica de decisão
if opcao == '1':
    destinatarios = emails_franquia
    texto_tipo = "Franquia"
    nome_do_arquivo = "RA.FRQ.xlsx"
    print(">> Configurando envio para FRANQUIA...")
elif opcao == '2':
    destinatarios = emails_lp
    texto_tipo = "L.P."
    nome_do_arquivo = "R.A-L.P.xlsx"
    print(">> Configurando envio para LOJAS PRÓPRIAS...")
else:
    print("Opção inválida! O script será encerrado para evitar erros.")
    sys.exit() # Encerra o programa se digitar algo errado

nome_arquivo_final = output / nome_do_arquivo
print(f"Gerando arquivo: {nome_arquivo_final}...")

with pd.ExcelWriter(nome_arquivo_final, engine='xlsxwriter') as writer: #pd.ExcelWriter: É como comprar uma pasta vazia e deixá-la aberta em cima da mesa. Ela serve para "agrupar" as folhas.
    #writer: É o nome que você deu para essa pasta enquanto ela está aberta na sua mão.
     #df.to_excel(writer...): É a ação de imprimir uma folha e colocá-la dentro dessa pasta específica, em vez de jogar a folha solta na mesa.
     #with: É um segurança automático. Ele garante que, assim que você terminar de colocar a última folha (ou se o prédio pegar fogo no meio do processo), a pasta será fechada, lacrada e guardada no armário corretamente.
    df_relatorio.to_excel(writer, sheet_name='Relatório', index=False) 
    TD_STK.to_excel(writer, sheet_name='ESTOQUE', index=False)
    TD_CANC.to_excel(writer, sheet_name='CANCELADO', index=False)
    TD_RESUMO.to_excel(writer, sheet_name='RESUMO', index=False)


print("Arquivo salvo com sucesso. Preparando e-mail...")

print("Iniciando envio de e-mail...")

usuario_login = "gabriel.thiago@ciahering.com.br"
email_remetente_visual = "relatorios.planejamento@ciahering.com.br"

# Credenciais
email_origem =  "relatorios.planejamento@ciahering.com.br"
senha_app = "sbmvcqubqmtvoxby" 
email_destino = ", ".join(destinatarios)

msg = EmailMessage()
# O Assunto agora é dinâmico usando a variável {texto_tipo}
msg['Subject'] = f"R.A 2.0 | {texto_tipo} 03.12 a 04.12"
msg['From'] = email_remetente_visual
msg['To'] =  email_destino

# O Corpo do e-mail agora também usa a variável {texto_tipo} dentro dos parênteses
corpo_html =f"""
<html>
    <body>
        <p>Olá, equipe!</p>
        <p>Segue abaixo o resumo das principais informações referentes ao abastecimento de R.A 2.0 desta semana ({texto_tipo}):</p>
        <ul>
            <li><b>Total de Necessidade:</b> {fmt_br(tot_necessidade)} unidades</li>
            <li><b>Total Sem Estoque:</b> {fmt_br(tot_sem_estoque)} unidades, o que representa <b>{pct_sem_estoque:.2f}%</b> da necessidade total</li>
            <li><b>Total Cancelado:</b> {fmt_br(tot_cancelado)} unidades, equivalente a <b>{pct_cancelado:.2f}%</b> da necessidade</li>
            <li><b>Receita Total:</b> R$ {fmt_br(tot_receita, eh_moeda=True)}, com volume atendido de {fmt_br(tot_atendido)} unidades</li>
        </ul>
        <p>Em anexo, estou enviando a listagem completa dos artigos sem estoque, assim como os cancelamentos abertos.</p>
        <p>Qualquer dúvida, fico à disposição.</p>
        <p>Atenciosamente,<br>
    </body>
</html>
"""
msg.set_content(corpo_html, subtype='HTML', charset='utf-8')

# 3. ANEXAR E ENVIAR
with open(nome_arquivo_final, 'rb') as f:  #"ler" o arquivo Excel antes de enviar. O 'rb' significa Read Binary (Leitura Binária)
    file_data = f.read() # #guardamos o conteúdo do arquivo na memória do computador
    file_name = nome_arquivo_final.name #Isso serve para que, quem receba o e-mail, veja apenas o nome do arquivo
    

msg.add_attachment(file_data, maintype='application', subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet', filename=file_name)

contexto = ssl.create_default_context() #Isso é segurança. O Gmail exige que a conexão seja criptografada.
try:
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=contexto) as smtp: #O SMTP (Simple Mail Transfer Protocol) é o protocolo padrão da internet para enviar e-mails.
        smtp.login(usuario_login, senha_app) 
        smtp.send_message(msg) 
        print(f"E-mail ({texto_tipo}) enviado com sucesso! De: {email_remetente_visual}")
except Exception as e:
    print(f"Erro ao enviar: {e}")


Selecione o tipo de envio:
1 - Franquia
2 - Lojas Próprias (L.P.)
>> Configurando envio para FRANQUIA...
Gerando arquivo: OUTPUT\RA.FRQ.xlsx...
>> Configurando envio para FRANQUIA...
Gerando arquivo: OUTPUT\RA.FRQ.xlsx...
Arquivo salvo com sucesso. Preparando e-mail...
Iniciando envio de e-mail...
Arquivo salvo com sucesso. Preparando e-mail...
Iniciando envio de e-mail...
E-mail (Franquia) enviado com sucesso! De: relatorios.planejamento@ciahering.com.br
E-mail (Franquia) enviado com sucesso! De: relatorios.planejamento@ciahering.com.br
